In [5]:
# Core imports
import os, sys, asyncio, yaml
from dotenv import load_dotenv
from pydantic import BaseModel
from contextlib import asynccontextmanager
# Project modules (assumes this notebook lives in src/mcp_client/)
from client import MCPClient
import json
import requests

In [6]:
import logging
import sys
from typing import Optional

DEFAULT_FORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"

def get_logger(
    name: str,
    log_file: Optional[str] = None,
    level: int = logging.INFO,
    console_level: Optional[int] = None,
    fmt: str = DEFAULT_FORMAT,
    propagate: bool = False,
):
    """Return a configured logger.

    Ensures we don't attach duplicate handlers if called multiple times.
    """
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.propagate = propagate

    formatter = logging.Formatter(fmt)

    # Add/ensure file handler
    if log_file:
        if not any(isinstance(h, logging.FileHandler) and getattr(h, 'baseFilename', None) and h.baseFilename.endswith(log_file) for h in logger.handlers):
            fh = logging.FileHandler(log_file)
            fh.setLevel(level)
            fh.setFormatter(formatter)
            logger.addHandler(fh)

    # Add/ensure console handler
    if console_level is None:
        console_level = level
    if not any(isinstance(h, logging.StreamHandler) and getattr(h, 'stream', None) is sys.stdout for h in logger.handlers):
        ch = logging.StreamHandler(sys.stdout)
        ch.setLevel(console_level)
        ch.setFormatter(formatter)
        logger.addHandler(ch)

    return logger

In [7]:
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY", "")

logger = get_logger("mcp-client", log_file="mcp_client.log", level=20, console_level=20)

from pathlib import Path
file_path = Path("/home/vmadmin/intent/src/test/anthropic_teste.csv")

In [8]:
# Create a client, connect, and process an intent (returns an OpenAI Responses API object)
client = MCPClient(logger=logger, rapp="http://10.233.7.113:8090", file_path= file_path)
connected = await client.connect_to_server("http://10.233.14.4:8000/sse")
await client.set_llm(api_key=api_key, llm_name="anthropic",llm_model="claude-sonnet-4-5")
if not connected:
    raise RuntimeError("Failed to connect to MCP server")

# # 'response' is an object (not a dict) with an 'output' attribute (a list of bl1ocks)
response = await client.process_intent(
   "Provision a slice for industrial motion control networks, requiring latency not exceeding 2 ms and uplink rates of 15 Mbps."
)

2026-01-28 18:23:08,805 - mcp-client - INFO - Attempting to connect to server at http://10.233.14.4:8000/sse.
2026-01-28 18:23:08,869 - mcp-client - INFO - Requesting MCP tools from the server.
2026-01-28 18:23:08,878 - mcp-client - INFO - Successfully connected to server. Available tools: ['create_session', 'get_session', 'delete_session', 'ping_api']
2026-01-28 18:23:08,879 - mcp-client - INFO - Setting llm anthropic - model claude-sonnet-4-5
2026-01-28 18:23:08,880 - mcp-client - INFO - Requesting MCP tools from the server.
2026-01-28 18:23:08,937 - mcp-client - INFO - LLM configuration completed successfully.
2026-01-28 18:23:08,939 - mcp-client - INFO - Calling LLM: anthropic
2026-01-28 18:23:12,543 - mcp-client - INFO - Assistant response: Message(id='msg_01HrAB3e9zZ4q4k9N3dhegsU', content=[TextBlock(citations=None, text="I'll create a network slice reservation for your industrial motion control network with the specified requirements.", type='text'), ToolUseBlock(id='toolu_017kW

In [9]:
print(response.usage_metadata.prompt_tokens_details[0])

AttributeError: 'str' object has no attribute 'usage_metadata'

In [ ]:
for cand in getattr(response, "candidates", []):
    